In [ ]:

import os
import time
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import mediapipe as mp
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models
from gtts import gTTS
from playsound import playsound

In [ ]:
tf.__version__
tf.test.is_gpu_available('gpu')

In [ ]:
input_types = ['palm', 'fist', 'thumbsup', 'gun', 'call']
description_types = ['01', '02', '03', '04', '05']

In [ ]:
cnn = models.load_model("cnn.h5")
cnn.summary()

In [ ]:
l = []
for i in range(5):
    image_matrix = plt.imread('Inputs/call/' + 'call_' + str(i + 1) + '.jpg')
    l.append(image_matrix)
l = np.array(l)
l = l / 255
cnn.predict(l)
l.shape

In [ ]:
def Predict(img):
    class_ = np.argmax(cnn.predict(img))
    return input_types[class_]

In [ ]:
Predict(np.array([l[0]]))

In [ ]:
def saving_sounds():
    language = 'en'
    for types in input_types:
        obj = gTTS(text=types, lang=language, slow=False)
        if os.path.isfile(str(types) + ".mp3") is False:
            obj.save(str(types) + ".mp3")

In [ ]:
saving_sounds()

In [ ]:
types = 'thumbsup'
playsound(str(types) + ".mp3")

In [ ]:
def playing_sounds():
    for types in input_types:
        playsound(str(types) + ".mp3")

In [ ]:
playing_sounds()

In [ ]:
def message_class(mirror1, class_):
    cv.putText(mirror1, str(class_), (100, 100), cv.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 1)
    return mirror1

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()
R = 25
thickness = 2
webcam = 0
capture = cv.VideoCapture(webcam)
pre_class = ''
fps = int(capture.get(cv.CAP_PROP_FPS))
print("fps is " + str(fps))
_, frame = capture.read()
height, width, channel = frame.shape

In [ ]:
while capture.isOpened():
    if cv.waitKey(1) & 0xFF == 13:
        break
    black = np.zeros(shape=frame.shape)
    _, frame = capture.read()
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    result = holistic.process(frame_rgb)
    try:
        hand_landmarks = result.right_hand_landmarks.landmark
        if hand_landmarks:
            x_max, y_max, x_min, y_min = 0, 0, width, height
            for i in range(21):
                lm = hand_landmarks[i]
                x, y = int(lm.x * width), int(lm.y * height)
                x_max, x_min = max(x, x_max), min(x, x_min)
                y_max, y_min = max(y, y_max), min(y, y_min)
            frame_bgr = cv.cvtColor(frame_rgb, cv.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(frame_bgr, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            cv.rectangle(frame_bgr, (x_min - R, y_min - R), (x_max + R, y_max + R), (0, 255, 0), thickness)
            result1 = frame_bgr
            mirror1 = cv.flip(result1, 1)
            mp_drawing.draw_landmarks(black, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            croped = black[y_min - R + thickness: y_max + R - thickness, x_min - R + thickness: x_max + R - thickness]
            resized = cv.resize(croped, (96, 96))
            mirror2 = cv.flip(resized, 1)
            result2 = mirror2
            img_mat = np.array([result2])
            class_ = Predict(img_mat)
            message_class(mirror1, class_)
            black1 = np.zeros(shape=frame.shape)
            cv.putText(black1, str(description_types[input_types.index(class_)]), (100, 100), cv.FONT_HERSHEY_PLAIN, 5, (255, 0, 0), 1)
            result3 = black1
            cv.imshow("Frame2", result2)
            cv.imshow("Frame3", result3)

In [ ]:
capture.release()
cv.destroyAllWindows()
